In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [5]:
vistas_por_area=vistas.copy()
vistas_por_area['vistas_por_area']=0;
vistas_por_area=pd.merge(vistas_por_area,avisos,on='idaviso', how='left')
vistas_por_area=vistas_por_area.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_por_area.head()

,idpostulante,nombre_area,vistas_por_area
0,0005E,Tecnologia / Sistemas,2
1,00Lkv,Comercio Exterior,2
2,00dMd,Comercio Exterior,4
3,01QAq,Contabilidad,1
4,021OM,Comercio Exterior,5


In [6]:
vistas_por_nivel=vistas.copy()
vistas_por_nivel['vistas_por_nivel']=0;
vistas_por_nivel=pd.merge(vistas_por_nivel,avisos,on='idaviso', how='left')
vistas_por_nivel=vistas_por_nivel.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})
vistas_por_nivel.head()

,idpostulante,nivel_laboral,vistas_por_nivel
0,0005E,Otro,2
1,00Lkv,Otro,2
2,00dMd,Otro,4
3,01QAq,Otro,1
4,021OM,Otro,5


In [7]:
vistas_por_tipo=vistas.copy()
vistas_por_tipo['vistas_por_tipo']=0;
vistas_por_tipo=pd.merge(vistas_por_tipo,avisos,on='idaviso', how='left')
vistas_por_tipo=vistas_por_tipo.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_por_tipo.head()

,idpostulante,tipo_de_trabajo,vistas_por_tipo
0,0005E,Full-time,2
1,00Lkv,Full-time,2
2,00dMd,Full-time,4
3,01QAq,Full-time,1
4,021OM,Full-time,5


In [8]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [9]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [10]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [11]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [12]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [13]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [14]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [15]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [16]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501993


In [17]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad
4481251,1112272863,Oqm6dEW,1,MASC,24.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,38.0,NaN,NaN
9478166,1112424032,8APRox,1,MASC,26.0,-,-,-,-,-,...,False,False,False,False,False,False,False,677.0,1.0,0.001477
13056659,1112489749,JBXZmde,0,MASC,34.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,58.0,NaN,NaN
4140201,1112316288,GNdjqDw,1,FEM,21.0,-,-,-,-,-,...,False,False,False,False,False,False,False,246.0,NaN,NaN
4262599,1112297756,rmRqPz1,0,MASC,53.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,37.0,NaN,NaN
7339652,1112344695,dY5mRRM,1,MASC,29.0,-,-,Graduado,-,Graduado,...,True,False,False,False,False,False,False,NaN,NaN,NaN
8232793,1112365322,X95NbbQ,0,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,9.0,NaN,NaN
14876243,1111424790,eG3E9B,0,MASC,49.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,7.0,NaN,NaN
4459760,1112404349,W9kaMb2,1,FEM,27.0,-,-,-,-,-,...,True,True,False,False,False,False,False,146.0,NaN,NaN
8178384,1111857740,mzbKVM9,0,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,6.0,NaN,NaN


In [18]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad,lo_vio
1012706,1112262301,mzVVeJZ,0,MASC,24.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,9.0,0.0,0.000000,False
3834570,1112114559,akjbjAq,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,1.0,0.0,0.000000,False
15295337,1112325605,owjoA4a,0,FEM,46.0,-,-,-,-,-,...,False,False,False,False,False,False,0.0,0.0,0.000000,False
14353151,1112190012,EbBqqz,0,FEM,34.0,-,-,-,-,-,...,False,False,False,True,False,True,15.0,0.0,0.000000,False
1369257,1112312278,88OZa3,1,FEM,33.0,-,-,-,-,-,...,False,False,False,False,True,False,12.0,0.0,0.000000,False
15568866,1111432986,5Q8zOZ,0,FEM,30.0,-,-,-,-,-,...,True,False,False,False,False,False,16.0,0.0,0.000000,False
2612592,1112257069,8xkGPD,0,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.0,0.0,0.000000,False
12556468,1112413624,ow2zJ9L,1,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,1.0,0.0,0.000000,False
15432743,1111918237,YwJaBX,0,FEM,48.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,3.0,0.0,0.000000,False
4133069,1112377265,6pA3KO,1,MASC,31.0,-,-,-,-,Graduado,...,True,False,False,False,True,False,23.0,4.0,0.173913,True


In [19]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Abastecimiento,1
1,0z5Dmrd,Administración,2
2,0z5Dmrd,Almacén / Depósito / Expedición,1
3,0z5Dmrd,Atención al Cliente,1
4,0z5Dmrd,Auditoría,1


In [20]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,-,1
1,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,2
2,0z5Dmrd,Junior,2
3,0z5Dmrd,Otro,1
4,0z5Dmrd,Senior / Semi-Senior,17


In [21]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,21
1,0z5Dmrd,Part-time,1
2,0z5Dmrd,Teletrabajo,1
3,0z5JW1r,Full-time,29
4,0z5JW1r,Part-time,1


In [22]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones=pd.merge(postulaciones,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
12776313,1112349430,vVjx9G1,0,FEM,53.0,-,-,-,-,-,...,149.0,0.0,0.000000,False,0,19,2,0.0,112.0,112.0
12160338,1112256054,1Qklv6B,0,FEM,22.0,-,-,-,-,Graduado,...,0.0,0.0,0.000000,False,0,0,12,0.0,0.0,0.0
7946430,1112357460,0zr8mLv,1,MASC,33.0,-,-,Graduado,-,Graduado,...,130.0,1.0,0.007692,True,81,319,219,18.0,127.0,127.0
10353188,1112502787,5mqX2Yz,0,MASC,24.0,-,-,-,-,Graduado,...,11.0,0.0,0.000000,False,15,34,29,7.0,10.0,10.0
11260593,1112341410,0zNPbeM,0,NO_DECLARA,200.0,-,-,-,-,-,...,6.0,0.0,0.000000,False,2,19,17,0.0,6.0,6.0
12899308,1112148951,6rPKr4O,0,MASC,18.0,-,-,-,-,Graduado,...,2.0,0.0,0.000000,False,1,18,15,0.0,1.0,1.0
1999261,1112406885,Nz03lZV,1,FEM,30.0,-,-,-,-,Graduado,...,4.0,0.0,0.000000,False,2,1,0,0.0,0.0,0.0
13688799,1112443841,LNK3RJP,1,FEM,30.0,-,-,-,-,Graduado,...,87.0,0.0,0.000000,False,20,168,141,8.0,72.0,72.0
14706909,1112427236,aLmaNq,0,MASC,31.0,-,-,-,-,Graduado,...,36.0,0.0,0.000000,False,0,36,22,0.0,32.0,32.0
14174472,1112193810,vVk1KA5,1,MASC,33.0,-,-,-,-,Graduado,...,0.0,0.0,0.000000,False,0,20,14,0.0,0.0,0.0


In [23]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0

Train:  15281861 Test:  804309


In [24]:
gc.collect()

312

In [25]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.686246
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.679643
[3]	valid_0's binary_logloss: 0.67284
[4]	valid_0's binary_logloss: 0.666009
[5]	valid_0's binary_logloss: 0.659385
[6]	valid_0's binary_logloss: 0.653093
[7]	valid_0's binary_logloss: 0.646785
[8]	valid_0's binary_logloss: 0.64044
[9]	valid_0's binary_logloss: 0.634356
[10]	valid_0's binary_logloss: 0.62851
[11]	valid_0's binary_logloss: 0.622508
[12]	valid_0's binary_logloss: 0.616593
[13]	valid_0's binary_logloss: 0.610852
[14]	valid_0's binary_logloss: 0.605708
[15]	valid_0's binary_logloss: 0.600451
[16]	valid_0's binary_logloss: 0.595544
[17]	valid_0's binary_logloss: 0.590112
[18]	valid_0's binary_logloss: 0.585185
[19]	valid_0's binary_logloss: 0.579927
[20]	valid_0's binary_logloss: 0.575107
[21]	valid_0's binary_logloss: 0.570043
[22]	valid_0's binary_logloss: 0.565108
[23]	valid_0's binary_logloss: 0.560572
[24]	valid_0's binary_logloss: 

In [26]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9785592779558807


In [27]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV26.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV26.pkl']

In [28]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['lo_vio']=(prediccion['cantidad']>0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion=pd.merge(prediccion,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
92774,91863,1112473259,8611zL,FEM,32.0,-,-,-,-,Graduado,...,45.0,NaN,0.000000,False,0.0,44.0,11.0,0.0,30.0,30.0
85754,35076,1112409877,YvG6V,MASC,55.0,-,-,-,-,Graduado,...,NaN,NaN,0.000000,False,0.0,17.0,0.0,0.0,0.0,0.0
64702,11631,1111962989,8ZlA8l,MASC,57.0,-,Graduado,-,-,Graduado,...,12.0,NaN,0.000000,False,0.0,17.0,15.0,3.0,12.0,12.0
13066,58553,1112452774,owERO4W,MASC,29.0,-,-,-,En Curso,Graduado,...,56.0,1.0,0.017857,True,1.0,36.0,25.0,2.0,46.0,46.0
22427,14644,1112094756,ow2V93Z,FEM,24.0,-,-,-,-,Graduado,...,51.0,1.0,0.019608,True,2.0,3.0,23.0,1.0,1.0,1.0
6523,84847,1112466576,VNrj6jD,FEM,26.0,-,-,-,-,-,...,136.0,2.0,0.014706,True,10.0,44.0,8.0,49.0,97.0,97.0
20042,80748,1112465210,PmGNx31,MASC,19.0,-,-,-,-,Graduado,...,171.0,1.0,0.005848,True,0.0,23.0,35.0,2.0,71.0,71.0
37224,57349,1112451292,EzEvGNN,MASC,25.0,-,-,-,-,Graduado,...,778.0,1.0,0.001285,True,7.0,701.0,494.0,8.0,683.0,683.0
25588,58387,1112452585,610YAR,FEM,28.0,-,-,-,-,Graduado,...,105.0,1.0,0.009524,True,33.0,118.0,79.0,23.0,86.0,86.0
57658,6798,1111667452,ZQAvpR,FEM,42.0,-,-,-,Graduado,Graduado,...,4.0,NaN,0.000000,False,0.0,17.0,13.0,0.0,4.0,4.0


In [29]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.98346292, 0.01653708],
       [0.96621483, 0.03378517],
       [0.97414185, 0.02585815],
       ...,
       [0.98911116, 0.01088884],
       [0.99234047, 0.00765953],
       [0.99567343, 0.00432657]])

In [30]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [31]:
prediccion['sepostulo']=y_final2

In [32]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,NaN,0.0,False,0.0,16.0,0.0,4.0,24.0,24.0,0.016537
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,NaN,0.0,False,5.0,62.0,0.0,0.0,6.0,6.0,0.033785
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,NaN,0.0,False,3.0,21.0,0.0,1.0,7.0,7.0,0.025858
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,NaN,0.0,False,1.0,19.0,1.0,0.0,0.0,0.0,0.003723
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,NaN,0.0,False,0.0,16.0,1.0,0.0,3.0,3.0,0.000653


In [33]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.016537
1,1,0.033785
2,2,0.025858
3,3,0.003723
4,4,0.000653


In [34]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion57.csv",index=False)

In [35]:
prediccion.sepostulo.mean()

0.46306104340892074